In [1]:
import os
import pandas as pd
import re
from scipy.signal import butter, filtfilt

In [11]:
import os
import pandas as pd
import re
from scipy.signal import butter, filtfilt

def process_workout_files(input_folder, output_folder, sampling_frequency):
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Get list of CSV files in the input folder
    files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

    for file_name in files:
        file_path = os.path.join(input_folder, file_name)
        df = pd.read_csv(file_path)

        # Step 1: Remove the first 1.5 seconds and the last 1.5 seconds
        if 'seconds_elapsed' in df.columns:
            max_time = df['seconds_elapsed'].max()
            df = df[(df['seconds_elapsed'] > 2) & (df['seconds_elapsed'] < (max_time - 1.5))]

        # Step 2: Keep only columns with "wristmotion" in the name or "seconds_elapsed"
        columns_to_keep = [col for col in df.columns if "wristmotion" in col.lower() or col.lower() == "seconds_elapsed"]
        df = df[columns_to_keep]

        # Step 3: Extract weight, set, reps, and exercise code from the file name
        weight_match = re.search(r'_W(\d+)_', file_name)
        set_match = re.search(r'_S(\d+)_', file_name)
        reps_match = re.search(r'_R(\d+)', file_name)
        exercise_code_match = re.search(r'_(.*?)_', file_name)

        weight = float(weight_match.group(1)) if weight_match else None
        set_number = float(set_match.group(1)) if set_match else None
        reps = float(reps_match.group(1)) if reps_match else None
        exercise_code = exercise_code_match.group(1) if exercise_code_match else None

        # Map exercise code to label number
        code_label_mapping = {
            "SBLP": 1, "CGCR": 2, "NGCR": 3, "SAP": 4, "MGTBR": 5, "AIDBC": 6, "MPBC": 7, 
            "SHC": 8, "SMS": 9, "LE": 10, "30DBP": 11, "DSP": 12, "DLR": 13, "SACLR": 14, 
            "MRF": 15, "FAPU": 16, "SBCTP": 17, "MSP": 18, "SECR": 19, "PUSH": 20, "PULL": 21, 
            "MTE": 22, "SHSS": 23, "STCR": 24, "ILE": 25, "CRDP": 26, "MIBP": 27, "APULL": 28, 
            "PREC": 29, "SSLHS": 30, "HT": 31, "SAOCTE": 32, "45DBP": 33, "SAODTE": 34, 
            "LHC": 35, "IDBC": 36, "DWC": 37, "CGOCTE": 38
        }
        label_number = code_label_mapping.get(exercise_code, None)

        # Add extracted columns to the DataFrame
        df['Weight'] = weight
        df['Set'] = set_number
        df['Reps'] = reps
        df['activity'] = exercise_code
        df['ActivityEncoded'] = label_number

        # Step 4: Convert specified columns to float
        columns_to_convert = [col for col in df.columns if "wristmotion" in col.lower()] + \
                             ['seconds_elapsed', 'Set', 'Reps', 'Weight']

        for col in columns_to_convert:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce')

        # Step 5: Apply Butterworth filter to "wristmotion" columns
        wristmotion_columns = [col for col in df.columns if "wristmotion" in col.lower()]
        if wristmotion_columns:
            # Define Butterworth filter parameters
            order = 7
            cutoff_frequency = 1  # Desired cutoff frequency in Hz
            nyquist_frequency = sampling_frequency / 2
            normalized_cutoff = cutoff_frequency / nyquist_frequency

            # Create Butterworth filter coefficients
            b, a = butter(order, normalized_cutoff, btype='low', analog=False)

            # Apply the filter to each wristmotion column
            for col in wristmotion_columns:
                df[col] = filtfilt(b, a, df[col].values)

        # Save the processed DataFrame
        output_file_path = os.path.join(output_folder, file_name)
        df.to_csv(output_file_path, index=False)
        print(f"Processed file saved: {output_file_path}")


In [13]:
# Example usage
input_folder = r"D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_Raw"
output_folder = r"D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered"
process_workout_files(input_folder, output_folder, sampling_frequency=100)


Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\011224_APULL_W61_S1_R12-2024-12-01_07-36-28.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\011224_APULL_W63_5_S2_R12-2024-12-01_07-41-19.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\011224_APULL_W66_S3_R10-2024-12-01_07-45-29.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\011224_APULL_W66_S4_R10-2024-12-01_07-49-19.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\011224_CGCR_W45_S1_R16-2024-12-01_07-53-55.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_Pro

C:\Users\shakt\AppData\Local\Temp\ipykernel_8856\3745315582.py:15: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\231124_45DBP_W12_5_S2_R16-2024-11-23_12-39-26.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\231124_SACLR_W4_5_S1_R18-2024-11-23_12-57-04.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\231124_SACLR_W4_5_S2_R18-2024-11-23_13-00-34.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\231124_SACLR_W4_5_S3_R18-2024-11-23_13-03-58.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\231124_SAOCTE_W11_S1_R15-2024-11-23_12-47-03.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales

C:\Users\shakt\AppData\Local\Temp\ipykernel_8856\3745315582.py:15: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\261124_30DBP_W17_5_S2_R12-2024-11-26_13-27-55.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\261124_45DBP_W12_5_S1_R12-2024-11-26_13-39-13.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\261124_CGOCTE_W18_S1_R12-2024-11-26_14-39-41.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\261124_MTE_W18_S1_R12-2024-11-26_14-34-13.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\261124_SACLR_W4_5_S1_R30-2024-11-26_13-57-49.csv


...

Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\261124_SACLR_W4_5_S2_R15-2024-11-26_14-06-07.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\261124_SAOCTE_W11_S1_R18-2024-11-26_13-45-55.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\261124_SAOCTE_W11_S2_R15-2024-11-26_13-51-14.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\261124_SAODTE_W7_5_S1_R16-2024-11-26_14-24-54.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\261124_SAODTE_W7_5_S2_R16-2024-11-26_14-30-57.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whale

C:\Users\shakt\AppData\Local\Temp\ipykernel_8856\3745315582.py:15: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\271124_CGCR_W52_S1_R15-2024-11-27_15-06-19.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\271124_DWC_W7_5_S1_R12-2024-11-27_15-49-31.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\271124_IDBC_W7_5_S1_R11-2024-11-27_15-21-34.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\271124_IDBC_W7_5_S2_R8-2024-11-27_15-25-35.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\271124_PREBC_W7_5_S1_R9-2024-11-27_15-30-47.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_Process

C:\Users\shakt\AppData\Local\Temp\ipykernel_8856\3745315582.py:15: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\291124_MTE_W18_S3_R13-2024-11-29_15-20-44.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\291124_SACLR_W4_5_S1_R30-2024-11-29_15-43-12.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\291124_SACLR_W4_5_S2_R15-2024-11-29_15-49-11.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\291124_SAOCTE_W11_S2_R11-2024-11-29_15-08-05.csv
Processed file saved: D:\My projects\Health Tracker Band\IMUWorkoutClassification\AppleWatchRecordings\Whales1_ProcessedFiltered\291124_SAOCTE_W18_S1_R7-2024-11-29_15-04-06.csv
